In [41]:
# !pip install openpyxl
# !pip install selenium
# !pip install webdriver_manager

In [42]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager # 크롬버전 상관없이 드라이버 설치 상관없이 
from selenium.webdriver.chrome.service import Service # 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
from openpyxl import load_workbook
from selenium.webdriver.chrome.options import Options

import pyperclip
import datetime
import time
import pandas as pd
import numpy as np
import csv

In [43]:
#-- browser options
# Options create
options = webdriver.ChromeOptions()
options.add_argument("incognito")   # secret mode
# options.add_argument("headless")   # background mode

In [44]:
#-- Activate webdriver 
# browser = webdriver.Chrome("../chromedriver.exe") # install driver mode
# browser = webdriver.Chrome(service=Service(ChromeDriverManager().install())) # Automatically installs the appropriate drive manager for your Chrome version.

# Browser with option applied 
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


In [45]:
#-- LogIn
def login_service(id,pwd):
    browser.get("https://nxlogin.nexon.com/common/login.aspx?redirect=https%3A%2F%2Fevents.fifaonline4.nexon.com%2F220908%2Fshoot%3Futm_source%3Dpc%26utm_medium%3DmainMiddle%26utm_campaign%3D220908_shoot")
    browser.implicitly_wait(1)
    search_id = browser.find_element(By.CSS_SELECTOR,"#txtNexonID")
    search_id.click()
    # search_id.send_keys(id)
    pyperclip.copy(id)
    search_id.send_keys(Keys.CONTROL,"v")
    time.sleep(1.4)
    search_pwd = browser.find_element(By.CSS_SELECTOR,"#txtPWD")
    search_pwd.click()
    # search_pwd.send_keys(pwd)
    pyperclip.copy(pwd)
    time.sleep(1.6)
    search_pwd.send_keys(Keys.CONTROL,"v")
    search_pwd.send_keys(Keys.ENTER)
    browser.implicitly_wait(1)
    time.sleep(1.5)
    try :    
        browser.find_element(By.CSS_SELECTOR,"#header > div.menu > button.btn_logout") 
        message = id + "\tLogIn Success"
    except:
        message = id + "\tLogIn Failed"
        print(message)
        browser.quit()
        
    print(message)
    
#-- LogOut
def logout(id):
    try:
        logout = browser.find_element(By.CSS_SELECTOR,"#header > div.menu > button.btn_logout")
        logout.click()
        browser.implicitly_wait(3)
        print(id,"\tLogOut complete")
    except NoSuchElementException as e : 
        print(id,"\tLogout button not found")
        browser.refresh()
        logout(id)
    time.sleep(1)
    
def refresh_btn():
    try:
        browser.refresh()
        rb = browser.find_element(By.CSS_SELECTOR,"#middle > section > div.summer33_wrap > div > div.header > div > div > button.btn_check")
        rb.send_keys(Keyd.ENTER)
        time.sleep(1.5)
    except ElementClickInterceptedException :
        print("ElementClickInterceptedException 발생")
        browser.refresh()
        refresh_btn()
    
    
def mission_data_upload(x,id):
    try:
        mission_list_name = [id]
        mission_list_data = [id]
        for i in range(1,7):
            mission_list_name.append(browser.find_element(By.CSS_SELECTOR,"#middle > section > div.aside > div > div > div:nth-child({}) > div.play > div.txt".format(str(i))).text.replace(" ",""))
            mission_list_data.append(browser.find_element(By.CSS_SELECTOR,"#middle > section > div.aside > div > div > div:nth-child({}) > div.play > div.state_txt > strong".format(str(i))).text.replace(" ","")+
                                    browser.find_element(By.CSS_SELECTOR,"#middle > section > div.aside > div > div > div:nth-child({}) > div.play > div.state_txt > span".format(str(i))).text.replace(" ",""))
        df.loc[2*x] = mission_list_name
        df.loc[2*x+1] = mission_list_data
        return df
    except (StaleElementReferenceException,ElementClickInterceptedException) as e :
        if e == StaleElementReferenceException:
            print("StaleElementReferenceException 발생")
        elif e == ElementClickInterceptedException :
            print("ElementClickInterceptedException 발생")
        else :
            print("new error 발생\n")
        browser.refresh()
        refresh_btn()
        mission_data_upload(x,id)

SyntaxError: '(' was never closed (1459647568.py, line 47)

In [ ]:
def file_create():
    dt_now = datetime.datetime.now()
    file_name = "mission_"+str(dt_now.date())
    return file_name.replace("-","_")

In [ ]:
#-- id / pwd load
from sqlite3 import complete_statement
id_data = pd.read_csv("../../../id_data.csv")

#-- data sheet create
id_count = id_data.shape[0]
mission_count = 7 # 6 + 1
df = pd.DataFrame(np.zeros([id_count*2,mission_count]))

# automations for website
for x in list(range(id_count)):
    d1 = id_data["id"][x]
    d2 = id_data["pwd"][x]
    
    login_service(d1,d2)
    refresh_btn()
    mission_data_upload(x,d1)
    logout(d1)

# Export data
df.to_excel("../../../{}.xlsx".format(file_create()))
d2= "clear"
df= "clear"
browser.quit()
print("complete")

ljh4176@gmail.com	LogIn Success
ljh4176@gmail.com 	LogOut complete
samj90s	LogIn Success
samj90s 	LogOut complete
calixlock@naver.com	LogIn Success
calixlock@naver.com 	LogOut complete
calixlock90@naver.com	LogIn Success
calixlock90@naver.com 	LogOut complete
amj90	LogIn Success


NameError: name 'ElementClickInterceptedException' is not defined